From [naturalscenesdataset.org](https://naturalscenesdataset.org):
```text
The Natural Scenes Dataset (NSD) is a large-scale fMRI dataset conducted at ultra-high-field (7T) strength at the Center of Magnetic Resonance Research (CMRR) at the University of Minnesota. The dataset consists of whole-brain, high-resolution (1.8-mm isotropic, 1.6-s sampling rate) fMRI measurements of 8 healthy adult subjects while they viewed thousands of color natural scenes over the course of 30–40 scan sessions. While viewing these images, subjects were engaged in a continuous recognition task in which they reported whether they had seen each given image at any point in the experiment. These data constitute a massive benchmark dataset for computational models of visual representation and cognition, and can support a wide range of scientific inquiry.
```


In [1]:
from cloudpathlib import S3Path, S3Client
from pathlib import Path

# Set up our cache path:
cache_path = Path('/tmp/cache')
if not cache_path.exists():
    cache_path.mkdir()

# Create the root S3Path for the NSD:
nsd_base_path = S3Path(
    's3://natural-scenes-dataset/',
    client=S3Client(
        no_sign_request=True,
        local_cache_dir=cache_path))

In [2]:
import os
import os.path as op
import glob
import nibabel as nib
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from nilearn import plotting

import urllib.request, zipfile
try:
    from pycocotools.coco import COCO
except ImportError as e:
    !pip install pycocotools
    from pycocotools.coco import COCO


#from nsd_access import NSDAccess

%matplotlib inline

In [3]:
from utils import ls, crawl

nsd_pppath = nsd_base_path / 'nsddata_betas' / 'ppdata'

ls(nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR')

[S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session01.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session02.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session03.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session04.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session05.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session06.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh.betas_session07.mgh'),
 S3Path('s3://natural-scenes-dataset/nsddata_betas/ppdata/subj01/fsaverage/betas_fithrf_GLMdenoise_RR/lh

### Loading data in volume space (1.8mm isotropic resolution)

In [4]:
vimg_filename = nsd_pppath / 'subj01' / 'func1pt8mm' / 'betas_fithrf_GLMdenoise_RR' / 'betas_session01.nii.gz'
vimg = nib.load(vimg_filename.fspath)
vimg = vimg.dataobj

# reshape to vector of nvoxels x ntimepoints
vimg = vimg.reshape([np.prod(vimg.shape[0:-1]),vimg.shape[-1]])

vimg.shape

[699192, 750]

### Loading data in surface space (fsaverage space)

In [5]:
simg_filename = nsd_pppath / 'subj01' / 'fsaverage' / 'betas_fithrf_GLMdenoise_RR' / 'lh.betas_session01.mgh'
simg = nib.load(simg_filename.fspath)
simg = simg.dataobj

# reshape to vector of nvoxels x ntimepoints
simg = simg.reshape([np.prod(simg.shape[0:-1]),simg.shape[-1]])

simg.shape

[163842, 750]

### Loading data in surface space (native surface space)

In [66]:
#ls(nsd_pppath / 'subj01' / 'nativesurface' / 'betas_fithrf_GLMdenoise_RR')

beta_session1 = nsd_pppath / 'subj01' / 'nativesurface' / 'betas_fithrf_GLMdenoise_RR' / 'lh.betas_session01.hdf5'
#227,021
beta_session1 = h5py.File(beta_session1.fspath,'r')

beta_session1=beta_session1['betas'][0] # this is just one trial

beta_session1.shape

(227021,)

### Visualization

In [9]:
#ls(nsd_base_path / 'nsddata' / 'freesurfer' / 'subj01' / 'label')

In [17]:
import neuropythy as ny

# Ask neuropythy to load a FreeSurfer subject:
sub = ny.freesurfer_subject('s3://natural-scenes-dataset/nsddata/freesurfer/subj01')

In [84]:
sorted(sub.lh.properties.keys())

['BA1_label',
 'BA1_weight',
 'BA2_label',
 'BA2_weight',
 'BA3a_label',
 'BA3a_weight',
 'BA3b_label',
 'BA3b_weight',
 'BA44_label',
 'BA44_weight',
 'BA45_label',
 'BA45_weight',
 'BA4a_label',
 'BA4a_weight',
 'BA4p_label',
 'BA4p_weight',
 'BA6_label',
 'BA6_weight',
 'DKT40_parcellation',
 'Desikan06_parcellation',
 'Destrieux09_parcellation',
 'MT_label',
 'MT_weight',
 'V1_label',
 'V1_weight',
 'V2_label',
 'V2_weight',
 'atlas_curvature',
 'brodmann_area',
 'brodmann_area_wide',
 'convexity',
 'cortex_label',
 'curvature',
 'entorhinal_label',
 'entorhinal_weight',
 'index',
 'jacobian_norm',
 'label',
 'midgray_surface_area',
 'parcellation',
 'perirhinal_label',
 'perirhinal_weight',
 'pial_curvature',
 'pial_surface_area',
 'surface_area',
 'thickness',
 'white_curvature',
 'white_surface_area']

In [19]:
ny.cortex_plot(sub.lh, surface='inflated', color=beta_session1)


Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…

### Masking with visual ROIs

In [95]:
beta_session1.shape == sub.lh.prop('V1_label').shape

True

In [118]:
LO = sub.lh.prop('Desikan06_parcellation')
LO = (LO == 11) | (LO == 13) | (LO == 21)

In [119]:
#masked_beta1 = beta_session1[sub.lh.prop('V1_label')]
masked_beta1 = beta_session1[LO]
masked_beta1.shape

(26664,)

In [120]:
ny.cortex_plot(sub.lh, surface='inflated', color=beta_session1, mask=LO)


Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=0.644570721372708, p…

### Dataframe setup

useful variables

In [75]:
n_sub = 8           # nr of subjects
n_sess = 8          # nr of sessions per subject
n_trial = 750       # nr of trials per session

sub_list =  [str(x) for x in np.arange(1,n_sub+1).tolist()]
sess_list = [str(x) for x in np.arange(1,n_sess+1).tolist()]
hem_list = ['lh','rh']

read betas and ROI masks 

In [126]:
def get_betas(sub_id,hem_id,sess_id,trial_nr):
    betas = nsd_pppath / ('subj0'+sub_list[sub_id]) / 'nativesurface' / 'betas_fithrf_GLMdenoise_RR' / (hem_list[hem_id]+'.betas_session0'+sess_list[sess_id]+'.hdf5')
    betas = h5py.File(betas.fspath,'r')
    betas = betas['betas'][trial_nr]
    
    
    mask = get_visual_mask(sub_id,hem_id)
    masked_betas = betas[mask]
    
    return betas, masked_betas

def get_visual_mask(sub_id,hem_id):
    sub = ny.freesurfer_subject('s3://natural-scenes-dataset/nsddata/freesurfer/subj0'+sub_list[sub_id])
    
    if hem_id == 0:
        desikan = sub.lh.prop('Desikan06_parcellation')
        mask = (desikan == 11) | (desikan == 13) | (desikan == 21)
    elif hem_id == 1:
        desikan = sub.rh.prop('Desikan06_parcellation')
        mask = (desikan == 11) | (desikan == 13) | (desikan == 21)
        
    return mask

In [127]:
sub_id = 0 
sess_id = 0
hem_id = 0 
trial_nr = 0 

betas, masked_betas = get_betas(sub_id,hem_id,sess_id,trial_nr)

In [ ]:
if hem_id == 0
    masked_betas = betas[sub.lh.prop('V1_label')]
elif hem_id == 1
    masked_betas = betas[sub.rh.prop('V1_label')]

In [42]:
stimdata_s1 = nsd_base_path / 'nsddata' / 'ppdata' / 'subj01' / 'behav' / 'responses.tsv'
#ls(stimdata_s1)
expdata = pd.read_table(stimdata_s1)
expdata

,SUBJECT,SESSION,RUN,TRIAL,73KID,10KID,TIME,ISOLD,ISCORRECT,RT,CHANGEMIND,MEMORYRECENT,MEMORYFIRST,ISOLDCURRENT,ISCORRECTCURRENT,TOTAL1,TOTAL2,BUTTON,MISSINGDATA
0,1,1,1,1,46003,626,0.505082,0,1.0,803.529781,0.0,NaN,NaN,0,1.0,1,0,1.0,0
1,1,1,1,2,61883,5013,0.505128,0,1.0,972.261383,0.0,NaN,NaN,0,1.0,1,0,1.0,0
2,1,1,1,3,829,4850,0.505175,0,1.0,742.351236,0.0,NaN,NaN,0,1.0,1,0,1.0,0
3,1,1,1,4,67574,8823,0.505221,0,1.0,747.518479,0.0,NaN,NaN,0,1.0,1,0,1.0,0
4,1,1,1,5,16021,9538,0.505267,0,1.0,547.422774,0.0,NaN,NaN,0,1.0,1,0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,40,12,58,13774,8984,262.629551,1,0.0,1275.300175,0.0,20963.0,21540.0,0,1.0,1,0,1.0,0
29996,1,40,12,59,66768,6026,262.629597,1,1.0,661.379768,0.0,16.0,17622.0,1,1.0,0,1,2.0,0
29997,1,40,12,60,53168,4841,262.629644,1,1.0,786.811781,0.0,9483.0,11912.0,0,0.0,0,1,2.0,0
29998,1,40,12,61,1944,7323,262.629690,1,1.0,502.626801,0.0,83.0,12162.0,1,1.0,0,1,2.0,0


In [44]:
sub_df = expdata[["SUBJECT","SESSION","RUN","TRIAL","73KID"]]
sub_df.head()

,SUBJECT,SESSION,RUN,TRIAL,73KID
0,1,1,1,1,46003
1,1,1,1,2,61883
2,1,1,1,3,829
3,1,1,1,4,67574
4,1,1,1,5,16021


In [48]:
min(sub_df['73KID'])

14